In [1]:
import pandas as pd
import numpy as np

import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis


import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
import pyLDAvis.gensim_models
import pandas as pd, numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
df=pd.read_csv('data_latest_without_any special characters_follow tweetss.csv')

In [5]:
df.columns

Index(['Unnamed: 0', 'user', 'date', 'tweets', 'user_location', 'tweetss'], dtype='object')

In [6]:
df=df[['user', 'date','user_location', 'tweetss']]

In [7]:
df

,user,date,user_location,tweetss
0,"{'username': 'LatebirdVeloz', 'id': 2545627424...",2019-01-07 23:58:19+00:00,Riviera Maya / CDMX ???Ã½???Â« ????,go hell bob parsons spineless fuck stop cowar...
1,"{'username': 'ANDREAG13353939', 'id': 87907853...",2019-01-07 23:56:23+00:00,"Colorado, USA",proudly presenting check let know think none g...
2,"{'username': 'GirlsinTech', 'id': 16014840, 'd...",2019-01-07 23:52:07+00:00,Global,want boss don t need go biz school entrepreneu...
3,"{'username': 'mark_callwood', 'id': 9057451792...",2019-01-07 23:46:48+00:00,Surrey,every cunty elephant killing bob parsons hundr...
4,"{'username': 'MccrelessTravis', 'id': 10715028...",2019-01-07 23:44:10+00:00,"Mackinaw city, michigan",proudly presenting check let know think gocent...
...,...,...,...,...
307115,"{'username': 'GoDaddyUK', 'id': 2511235328, 'd...",2022-09-23 07:50:12+00:00,UK,era advancing technology everevolving ways wor...
307116,"{'username': 'ecarlesi', 'id': 19444180, 'disp...",2022-09-23 07:50:33+00:00,"Roma, Lazio",possible threat phishing
307117,"{'username': 'ecarlesi', 'id': 19444180, 'disp...",2022-09-23 07:55:09+00:00,"Roma, Lazio",possible threat
307118,"{'username': 'ARCH1V3S23', 'id': 1498900982209...",2022-09-23 08:04:14+00:00,"she/they, minor, works on Ã¢ÂÂ¡Ã¯Â¸Â",adobe express u nawt hv layer something like c...


In [4]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [5]:
import nltk
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
review_data=df.copy()

In [11]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307120 entries, 0 to 307119
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user           307120 non-null  object
 1   date           307120 non-null  object
 2   user_location  307120 non-null  object
 3   tweetss        307120 non-null  object
dtypes: object(4)
memory usage: 9.4+ MB


In [6]:
review_data.dropna(axis = 0, how ='any',inplace=True) 



review_data['tweets'] = review_data['tweetss'].apply(clean_text)
review_data['Num_words_text'] = review_data['tweetss'].apply(lambda x:len(str(x).split())) 

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]

print('No of Short reviews')
print(len(df_short_reviews))



NameError: name 'review_data' is not defined

In [13]:
df_short_reviews

,user,date,user_location,tweetss,tweets,Num_words_text
8,"{'username': 'ItsikRomano', 'id': 57819532, 'd...",2019-01-07 23:26:54+00:00,Montreal - Quebec - Canada,sell contact info spammers india since purcha...,sell contact info spammers india since purchas...,26
30,"{'username': 'LaDonaHarvey', 'id': 225056299, ...",2019-01-07 22:14:10+00:00,San Diego,happen last time registered domain name phone ...,happen last time registered domain name phone ...,20
47,"{'username': 'maryderrickart', 'id': 562518183...",2019-01-07 21:28:03+00:00,Canada,disgusting pos kills fun founder time boycott...,disgusting kills founder time boycott ahole kn...,26
58,"{'username': '_STD_', 'id': 480888266, 'displa...",2019-01-07 20:59:56+00:00,Nor Cal,mmanerd top three things mentioned suffice wor...,mmanerd three things mentioned suffice wordpre...,20
65,"{'username': 'LonTaylor7', 'id': 1045821487876...",2019-01-07 20:30:52+00:00,"Newport, NJ",greatest plague since dawn time narcissistic ...,greatest plague since dawn time narcissistic p...,20
...,...,...,...,...,...,...
307096,"{'username': 'OhINoSheTwitent', 'id': 13933296...",2022-09-23 02:56:49+00:00,World,quentin latham one confirmed via abuses ts mad...,quentin latham confirmed abuses made clear fed...,27
307100,"{'username': 'codebeastmode', 'id': 9650420613...",2022-09-23 04:05:05+00:00,"Tampa, FL",negative whos driving many votes take agree w...,negative whos driving many votes take agree we...,22
307104,"{'username': 'TheAdamKingShow', 'id': 15649842...",2022-09-23 04:51:11+00:00,Los Angeles,check newly updated webpage we ve added ad...,check newly updated webpage added additional p...,24
307107,"{'username': 'SmokingGun', 'id': 192824056, 'd...",2022-09-23 05:16:03+00:00,"Perth, Western Australia",harmful effects week trespassing traps queen e...,harmful effects week trespassing traps queen e...,25


In [14]:
df_sampled=df_short_reviews.copy()

In [7]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_sampled['tweetss']=df_sampled['tweetss'].apply(remove_stopwords)


NameError: name 'df_sampled' is not defined

In [10]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
       output = []
       for sent in texts:
             doc = nlp(sent) 
             output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
       return output

In [8]:
#df_sampled.to_csv('df_sampled_rough.csv')
df_sampled=pd.read_csv('df_sampled_rough.csv')

In [11]:
text_list=df_sampled['tweetss'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

happen last time registered domain name phone call phone call phone call may change damned number spitting nailsomg 
['last', 'time', 'domain', 'name', 'phone', 'call', 'phone', 'call', 'phone', 'call', 'number', 'nailsomg']


In [12]:
print(tokenized_reviews[2])

['disgusting', 'fun', 'founder', 'time', 'boycott', 'ahole', 'd', 'love', 'jail', 'innocent', 'beautiful', 'animal', 'fun', 'camera', 'asshole', 'amp', 'picture']


In [13]:
#Create vocabulary dictionary and document term matrix
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [14]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)

In [33]:
lda_model.print_topics()  #Topics with their keywords (scored from highest to lowest)

[(0,
  '0.029*"size" + 0.022*"system" + 0.021*"solution" + 0.020*"r" + 0.019*"large" + 0.015*"share" + 0.014*"whole" + 0.013*"world" + 0.011*"career" + 0.010*"man"'),
 (1,
  '0.067*"provider" + 0.048*"detail" + 0.029*"order" + 0.029*"e" + 0.026*"cpanel" + 0.021*"similar" + 0.019*"data" + 0.015*"wordpress" + 0.013*"button" + 0.012*"hostgator"'),
 (2,
  '0.102*"social" + 0.099*"medium" + 0.047*"amp" + 0.026*"video" + 0.020*"right" + 0.020*"life" + 0.018*"comhow" + 0.012*"step" + 0.012*"available" + 0.010*"recent"'),
 (3,
  '0.034*"customer" + 0.031*"service" + 0.029*"time" + 0.022*"bigrock" + 0.021*"support" + 0.019*"day" + 0.018*"account" + 0.018*"issue" + 0.018*"website" + 0.015*"company"'),
 (4,
  '0.123*"business" + 0.027*"online" + 0.027*"brand" + 0.021*"small" + 0.019*"rare" + 0.017*"startup" + 0.017*"platform" + 0.016*"owner" + 0.015*"market" + 0.014*"top"'),
 (5,
  '0.271*"domain" + 0.068*"name" + 0.031*"registration" + 0.025*"com" + 0.022*"year" + 0.019*"registrar" + 0.017*"chea

In [16]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.265866  0.113036       1        1  22.999550
5     -0.139753 -0.387260       2        1  16.001446
7     -0.131099  0.034774       3        1  13.420542
6     -0.191686  0.135716       4        1  11.505434
8     -0.044244  0.049081       5        1   8.083961
4      0.059101  0.020382       6        1   7.455055
9      0.174434  0.039901       7        1   5.953630
0      0.180715  0.029579       8        1   5.799477
2      0.166993 -0.078085       9        1   5.740885
1      0.191404  0.042876      10        1   3.040021, topic_info=           Term          Freq         Total Category  logprob  loglift
3        domain  20500.000000  20500.000000  Default  30.0000  30.0000
292       email   5631.000000   5631.000000  Default  29.0000  29.0000
78     business   4344.000000   4344.000000  Default  28.0000  28.0000
68          web   5108.000000   5108.000000  Default  27.0000  27.0000
8          name   5146.000000   5146.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1852      brain     85.542884     86.462756  Topic10  -5.1224   3.4826
2943          n     83.406320     84.326127  Topic10  -5.1477   3.4823
8807      estas     82.802363     83.722186  Topic10  -5.1550   3.4823
433   wordpress    220.155756   4040.084449  Topic10  -4.1771   0.5836
69      website    108.244211   6601.124863  Topic10  -4.8870  -0.6173

[448 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
120       1  0.887717     able
120       4  0.111515     able
377       5  0.994880    abuse
434       1  0.847855   access
434       4  0.150622   access
...     ...       ...      ...
70        1  0.391318     year
70        2  0.456764     year
70        4  0.097898     year
70        5  0.053721     year
2458      6  0.993876  youtube

[519 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 8, 7, 9, 5, 10, 1, 3, 2])

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.376410990924025


In [19]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=tokenized_reviews, start=2, limit=50, step=1)

In [ ]:
# Show graph
limit=50; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()# Print the coherence scores

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[7]
model_topics = optimal_model.show_topics(formatted=False)
optimal_model.print_topics(num_words=10)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model, doc_term_matrix, dictionary)
vis